In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train_dir = '/content/drive/MyDrive/10 monkey/training/training'
test_dir = '/content/drive/MyDrive/10 monkey/validation/validation'

In [3]:
cols = ['Label', 'Latin Name', 'Common Name', 'Train Images', 'Validation Images']
df = pd.read_csv('/content/drive/MyDrive/10 monkey/monkey_labels.txt', names=cols, skiprows=1)

In [4]:
df.head()

,Label,Latin Name,Common Name,Train Images,Validation Images
0,n0,alouatta_palliata\t,mantled_howler,131,26
1,n1,erythrocebus_patas\t,patas_monkey,139,28
2,n2,cacajao_calvus\t,bald_uakari,137,27
3,n3,macaca_fuscata\t,japanese_macaque,152,30
4,n4,cebuella_pygmea\t,pygmy_marmoset,131,26


In [5]:

from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential

In [6]:
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

In [7]:
IMAGE_SIZE = [100, 100]

In [8]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58892288/58889256 [==============================] - 1s 0us/step


In [9]:
for layer in vgg.layers:
  layer.trainable = False

In [10]:
folders = glob('/content/drive/MyDrive/10 monkey/training/training/*')

In [11]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)

In [12]:
model = Model(inputs=vgg.input, outputs=prediction)

In [13]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 100, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0     

In [14]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [15]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)


In [16]:
training_set = train_datagen.flow_from_directory(train_dir,
                                                 target_size = (100, 100),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(test_dir,
                                            target_size = (100, 100),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 1145 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [17]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=25
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
36/36 [==============================] - 488s 13s/step - loss: 2.1636 - accuracy: 0.2368 - val_loss: 1.6111 - val_accuracy: 0.4559
Epoch 2/25
36/36 [==============================] - 33s 941ms/step - loss: 1.4261 - accuracy: 0.5272 - val_loss: 1.3419 - val_accuracy: 0.5441
Epoch 3/25
36/36 [==============================] - 33s 933ms/step - loss: 1.1094 - accuracy: 0.6694 - val_loss: 1.2040 - val_accuracy: 0.5919
Epoch 4/25
36/36 [==============================] - 33s 932ms/step - loss: 1.0052 - accuracy: 0.6832 - val_loss: 1.0884 - val_accuracy: 0.6544
Epoch 5/25
36/36 [==============================] - 33s 919ms/step - loss: 0.7953 - accuracy: 0.7843 - val_loss: 1.0404 - val_accuracy: 0.6654
Epoch 6/25
36/36 [==============================] - 33s 925ms/step - loss: 0.8271 - accuracy: 0.7692 - val_loss: 0.9878 - val_accuracy: 0.7096
Epoch 7/25
36/36 [==============================] - 33s 919ms/step - loss: 0.6993 - accuracy: 0.8063 - val_loss: 1.0085 - val_accuracy: 0.6691


In [18]:
from keras.models import load_model

model.save('VGG16.h5')